In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import time
import imp
import os
import os, os.path
import datetime as dt
import dateutil.parser as dp
import dill

%matplotlib inline

In [ ]:
dill.load_session('notebook_env_all_3_dfs.db')

print('attr_df.shape = ', attr_df.shape)
print('geo_df.shape = ', geo_df.shape)
print('seller_df.shape = ', seller_df.shape)

df = attr_df.merge(geo_df,on=['cusip','asofdate'])
df = df.drop('Label_y',axis=1)
df = df.drop(['poolno_y','poolno_x','Label_x'],axis=1)
seller_df = seller_df.drop('Label',axis=1)
df = df.merge(seller_df,on=['cusip','asofdate'])
df = df.drop(['Prefix_y','Prefix_x'],axis=1)

print('df.shape = ', df.shape)

In [ ]:
df.head()

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
df.shape

In [ ]:
len(df[df['cnloans']>=1500])

In [ ]:
# nloans = 250
# ddf = df[df['cnloans']>=nloans]

# rows = ddf.shape[0]

# N = rows//30000 + 1

# for i in range(N):
#     ddf.iloc[i*30000:(i+1)*30000,].to_csv(f'data/pools_ge{nloans}_loans_part{i+1}.csv')

In [ ]:
# nloans = 250
# df[df['cnloans']>=nloans].to_csv(f'pools_ge{nloans}_loans.csv',index=False)

# nloans = 500
# df[df['cnloans']>=nloans].to_csv(f'pools_ge{nloans}_loans.csv',index=False)

# nloans = 750
# df[df['cnloans']>=nloans].to_csv(f'pools_ge{nloans}_loans.csv',index=False)

# nloans = 1000
# df[df['cnloans']>=nloans].to_csv(f'pools_ge{nloans}_loans.csv',index=False)

# nloans = 1500
# df[df['cnloans']>=nloans].to_csv(f'pools_ge{nloans}_loans.csv',index=False)

# nloans = 2000
# df[df['cnloans']>=nloans].to_csv(f'pools_ge{nloans}_loans.csv',index=False)

# nloans = 2500
# df[df['cnloans']>=nloans].to_csv(f'pools_ge{nloans}_loans.csv',index=False)

In [ ]:
len(df[df['cnloans']>1000])

In [ ]:
df['Seasonality'] = df['asofdate'].apply(lambda x: int(str(x)[-2:]))

In [ ]:
import datetime as dt

df['asofdate'] = df['asofdate'].apply(lambda x: dt.date(int(str(x)[:4]),int(str(x)[-2:]),1))
df['asofdate'].head()

In [ ]:
simple_model_columns = ['SMM',
                        'ONLoans','cnloans',
                        'spread',
                        'DayCount','Seasonality','factor',
                        'Wac','Wam','Age',
                        'aols','waols',
                        'CSato',
                        'fico',
                        'cltv','ccltv',
                        'dti',
                        'PurpPct_purchase','PurpPct_refi',
                        'PctChannel_Broker','PctChannel_Corr','PctChannel_Retail',
                        'OccPct_investor','OccPct_owner','PropUnitsPct_2-4',
                        
                        'StatePct_AK','StatePct_AL','StatePct_AR','StatePct_AZ','StatePct_CA','StatePct_CO','StatePct_CT',
                        'StatePct_DC','StatePct_DE','StatePct_FL','StatePct_GA','StatePct_GU','StatePct_HI','StatePct_IA',
                        'StatePct_ID','StatePct_IL','StatePct_IN','StatePct_KS','StatePct_KY','StatePct_LA','StatePct_MA',
                        'StatePct_MD','StatePct_ME','StatePct_MI','StatePct_MN','StatePct_MO','StatePct_MS','StatePct_MT',
                        'StatePct_NC','StatePct_ND','StatePct_NE','StatePct_NH','StatePct_NJ','StatePct_NM','StatePct_NV',
                        'StatePct_NY','StatePct_OH','StatePct_OK','StatePct_OR','StatePct_PA','StatePct_PR','StatePct_RI',
                        'StatePct_SC','StatePct_SD','StatePct_TN','StatePct_TX','StatePct_UT','StatePct_VA','StatePct_VI',
                        'StatePct_VT','StatePct_WA','StatePct_WI','StatePct_WV','StatePct_WY',
                        
                        'SellerPct_AMRHT','SellerPct_ALS','SellerPct_CAFULL','SellerPct_CNTL','SellerPct_CITIZ',
                        'SellerPct_53','SellerPct_FIR','SellerPct_FRDOM','SellerPct_GUILD','SellerPct_CHASE',
                        'SellerPct_LLSL','SellerPct_MATRX','SellerPct_NCM','SellerPct_NATIONST','SellerPct_NRESM',
                        'SellerPct_PNYMAC','SellerPct_PILOSI','SellerPct_QUICK','SellerPct_REG','SellerPct_RMSC',
                        'SellerPct_UNSHFI','SellerPct_WFHM']

dfm = df[simple_model_columns]

dfm = dfm[dfm['cnloans']>=250]

#dfm = dfm[dfm['oltv']<=80]
len(dfm)

In [ ]:
dfm.dropna(inplace=True)

dfm = dfm.drop(['ONLoans','cnloans'],axis=1)

In [ ]:
dummies = pd.get_dummies(dfm['Seasonality'],drop_first=True)
dfm = dfm.drop('Seasonality',axis=1)
dfm = pd.concat([dfm,dummies],axis=1)

dfm.head()

In [ ]:
from sklearn.model_selection import train_test_split

X = dfm.drop('SMM',axis=1)
y = dfm['SMM']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

X_train.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout

In [ ]:
model = Sequential()

#input layer / first hidden layer
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
#model.add(Dropout(0.3))
model.add(Dropout(0.5))

# hidden layers
model.add(Dense(256, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
# model.add(Dropout(0.5))

#output layer
model.add(Dense(1)) # no activation means linear activation a(x)=x

model.compile(optimizer='adam', loss='mse')
#model.compile(optimizer='rmsprop', loss='mse')

In [ ]:
model.fit(x=X_train,
          y=y_train.values,
          validation_data=(X_test,y_test.values),
          batch_size=1024,
          epochs=130)

In [ ]:
losses = pd.DataFrame(model.history.history)

losses.plot(figsize=(16,4),grid=True)

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score

In [ ]:
test_predictions = model.predict(X_test)

In [ ]:
mean_absolute_error(y_test,test_predictions)

In [ ]:
np.sqrt(mean_squared_error(y_test,test_predictions))

In [ ]:
explained_variance_score(y_test,test_predictions)

In [ ]:
plt.scatter(y_test,test_predictions)

plt.plot(y_test,y_test,'r'); plt.grid()

In [ ]:
train_predictions = model.predict(X_train)

In [ ]:
mean_absolute_error(y_train,train_predictions)

In [ ]:
np.sqrt(mean_squared_error(y_train,train_predictions))

In [ ]:
explained_variance_score(y_train,train_predictions)

In [ ]:
plt.scatter(y_train,train_predictions)

plt.plot(y_train,y_train,'r'); plt.grid()

In [ ]:
dfm_with_predictions = dfm
dfm_with_predictions.head()

In [ ]:
dfm_with_predictions['model SMM'] = model.predict(scaler.transform(dfm_with_predictions.drop('SMM',axis=1)))

In [ ]:
dfm_with_predictions = dfm_with_predictions.join(df[['asofdate','cusip','CBal','Coupon']])

In [ ]:
dfm_with_predictions.head()

In [ ]:
dfm_with_predictions['CBal'] = dfm_with_predictions['CBal'].apply(lambda x: float(x))

In [ ]:
dfm_with_predictions['SMM x CBal'] = dfm_with_predictions.apply(lambda x: x['SMM']*x['CBal'],axis=1)
dfm_with_predictions['model SMM x CBal'] = dfm_with_predictions.apply(lambda x: x['model SMM']*x['CBal'],axis=1)

In [ ]:
dfm_with_predictions.head()

In [ ]:
temp = dfm_with_predictions[['Coupon','asofdate','SMM x CBal','model SMM x CBal','CBal']]

In [ ]:
temp = temp.groupby(['Coupon','asofdate']).sum()

temp['total SMM'] = temp['SMM x CBal']/temp['CBal']
temp['total model SMM'] = temp['model SMM x CBal']/temp['CBal']

temp['CPR'] = 100*(1-(1-temp['total SMM']/100)**12)
temp['model CPR'] = 100*(1-(1-temp['total model SMM']/100)**12)

In [ ]:
temp.head()

In [ ]:
for i, row in temp.iterrows():    
    print(f"Prepay month = {i} | CPR = {np.round(row['CPR'],1)} | Model CPR = {np.round(row['model CPR'],1)}")

In [ ]:
fig = plt.figure(figsize=(16,16))

for i in range(8):
    ax = fig.add_subplot(8,1,i+1)
    coupon = 2.5+i*0.5
    temp[['CPR','model CPR']].unstack(level=0)[[('CPR',coupon),('model CPR',coupon)]].plot(ax=ax,grid=True)